<a href="https://colab.research.google.com/github/QuanRy/TeleBot_VSTU_info_New_version_V2/blob/Samorokov_N_branch/model_VSTU_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#БИБЛИОТЕКИ
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import matplotlib as mpl
import pandas as pd
import numpy as np
import sklearn
import time
import sys
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:

# Загрузка данных
df = pd.read_csv("Russian_QA_Jeopardy_dataset_extended.csv",delimiter='\t')
df.head()

,QuestionID,QuestionNumber,QuestionText,Answer,Topic,Source,Author Full Name,TourName,TourLink
0,0,1,"Скажите по-японски ""короткая песня"".",Танка,В ПЕРЕВОДЕ,None,Юрий Гришов,"""Своя игра"". Юрий Гришов, 1999-2003",https://db.chgk.info/tour/grishov
1,1,2,"Если вы вспомните историю Германии, то, возмож...",Аншлюс,В ПЕРЕВОДЕ,None,Юрий Гришов,"""Своя игра"". Юрий Гришов, 1999-2003",https://db.chgk.info/tour/grishov
2,2,3,ЭТОТ тип письма в переводе с греческого означа...,Бустрофедон,В ПЕРЕВОДЕ,None,Юрий Гришов,"""Своя игра"". Юрий Гришов, 1999-2003",https://db.chgk.info/tour/grishov
3,3,4,В школе вам ставили оценки за поведение. А как...,Кондуит,В ПЕРЕВОДЕ,None,Юрий Гришов,"""Своя игра"". Юрий Гришов, 1999-2003",https://db.chgk.info/tour/grishov
4,4,5,"Относительно своего отца ОН действительно ""сра...",Телемах,В ПЕРЕВОДЕ,None,Юрий Гришов,"""Своя игра"". Юрий Гришов, 1999-2003",https://db.chgk.info/tour/grishov


In [ ]:
# Удаление столбцов
df.drop(columns=['QuestionID',	'QuestionNumber', 'TourLink', 'TourName', 'Author Full Name', 'Topic', 'Source'], inplace=True)

In [ ]:
df.head(0-10)

,QuestionText,Answer
0,"Скажите по-японски ""короткая песня"".",Танка
1,"Если вы вспомните историю Германии, то, возмож...",Аншлюс
2,ЭТОТ тип письма в переводе с греческого означа...,Бустрофедон
3,В школе вам ставили оценки за поведение. А как...,Кондуит
4,"Относительно своего отца ОН действительно ""сра...",Телемах
...,...,...
29361,В 2011 году ЕМУ был установлен памятник с табл...,Иван Поддубный
29362,Первый христианский монарх Экваториальной Афри...,Жуан {зачет: Жуау}
29363,"Эротическую поэму ""Дом Дианы"" ОН написал в 133...",Джованни Боккаччо
29364,Оригинальное название ЭТОГО ФИЛЬМА — отсылка к...,"Зомби по имени Шон {зачет: ""Shaun of the dead""}"


In [ ]:
# Извлечение вопросов и ответов
questions = df['QuestionText'].tolist()
answers = df['Answer'].tolist()

In [ ]:
# Инициализация токенизатора
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)

In [ ]:
# Преобразование текста в числовые последовательности
question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

# Определение максимальной длины последовательности
max_seq_length = max(len(seq) for seq in question_sequences + answer_sequences)

# Добавление заполнения для последовательностей одинаковой длины
question_sequences = pad_sequences(question_sequences, maxlen=max_seq_length, padding='post')
answer_sequences = pad_sequences(answer_sequences, maxlen=max_seq_length, padding='post')

In [ ]:
# Создание модели
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
hidden_units = 256


#СОЗДАНИЕ МОДЕЛИ
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=16, activation='relu', input_shape=[X.shape[1]]),
    tf.keras.layers.Dense(units=8, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

In [ ]:
# Компиляция модели
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Разделение данных на обучающую и тестовую выборки
train_size = int(0.8 * len(question_sequences))
train_questions = question_sequences[:train_size]
train_answers = answer_sequences[:train_size]
test_questions = question_sequences[train_size:]
test_answers = answer_sequences[train_size:]

#РАЗДЕЛЕНИЕ ДАННЫХ НА ТРЕНИРОВОЧНЫЙ И ТЕСТОВЫЙ НАБОРЫ
train_questions, test_questions, train_answers, test_answers = train_test_split(question_sequences, answer_sequences, test_size=0.2)

In [ ]:
# Обучение модели
num_epochs = 10
batch_size = 64


# train_questions = train_questions.toarray()
# test_questions = test_questions.toarray()

# tf.reshape(train_questions, [-1])
# tf.reshape(test_questions, [-1])
# tf.reshape(train_answers, [-1])
# tf.reshape(test_answers, [-1])

# model.add(tf.keras.layers.Flatten(input_shape=(132,132,3))) #this line

model.fit(train_questions, train_answers, epochs=num_epochs, batch_size=batch_size, validation_data=(test_questions, test_answers))

Epoch 1/10


ValueError: ignored